In [2]:
# the standalone script to identify and classfy_nodel from Ningbo Data
# full_prep
from preprocessing import full_prep
from config_submit import config as config_submit

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
from layers import acc

from data_detector import DataBowl3Detector,collate
from data_classifier import DataBowl3Classifier

from utils import *
from split_combine import SplitComb
from test_detect import test_detect
from importlib import import_module
import pandas
import numpy as np

datapath = config_submit['datapath']
prep_result_path = config_submit['preprocess_result_path']
skip_prep = True
skip_detect = False

if not skip_prep:
    print('It will take ups to 2min to do the preprocess, you will be able to skip it in a second run')
    testsplit, sliceim, extendbox = full_prep(datapath,prep_result_path,
                          n_worker = config_submit['n_worker_preprocessing'],
                          use_existing=config_submit['use_exsiting_preprocessing'])
    print('region of interest shape:',sliceim.shape)
    print('extendbox shape defined:',extendbox)
else:
    testsplit = os.listdir(datapath)
print('Loading QED nodule detection Master')
nodmodel = import_module(config_submit['detector_model'].split('.py')[0])
config1, nod_net, loss, get_pbb = nodmodel.get_model()
checkpoint = torch.load(config_submit['detector_param'])
nod_net.load_state_dict(checkpoint['state_dict'])

#torch.cuda.set_device(0)
#nod_net = nod_net.cuda()
#cudnn.benchmark = True
# Try working with CPU

nod_net = DataParallel(nod_net)

bbox_result_path = './bbox_result'
if not os.path.exists(bbox_result_path):
    os.mkdir(bbox_result_path)
testsplit = [f.split('_clean')[0] for f in os.listdir(prep_result_path) if '_clean' in f]
print testsplit
config1['crop_size'] = [32,32,32]
if not skip_detect:
    margin = 32
    sidelen = 144
    config1['datadir'] = prep_result_path
    split_comber = SplitComb(sidelen,config1['max_stride'],config1['stride'],margin,pad_value= config1['pad_value'])
    dataset = DataBowl3Detector(testsplit,config1,phase='test',split_comber=split_comber)
    test_loader = DataLoader(dataset,batch_size = 1,
        shuffle = False,num_workers = 8,pin_memory=False,collate_fn =collate)
    print 'pk'
    test_detect(test_loader, nod_net, get_pbb, bbox_result_path,config1,n_gpu=config_submit['n_gpu'])

Loading QED nodule detection Master
['8b9a2378b94d14526f266148c7991e78f20fd1f4', '06ea1ff317a8652bb74f6710efa3b758be96524e', '8a5a850d1f90a3b5e4cb6a3a137ac6be0c6768d9', 'ce1ff1a4-180d882f-5401a107-db5428b4-f58b0322', '6e0d4bef9a6a3c88d1c24cbb4e64d26526ad7cb1', 'ded4983b-85e8b6b0-9b9c0898-232b7d47-c2122a5d', 'c5aeb59f-9b52b436-4c12187a-fd90406d-fb55cc4b', '000014.mhd']
pk
torch.Size([8, 1, 208, 208, 208])


RuntimeError: torch.cuda.FloatTensor is not enabled.

In [1]:
import argparse
import os
import time
import numpy as np
from importlib import import_module
import shutil
from utils import *
import sys
from split_combine import SplitComb

import torch
from torch.nn import DataParallel
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

from layers import acc



net = nod_net
save_dir = bbox_result_path
config = config1
n_gpu = 1
start_time = time.time()
data_loader = test_loader
data_loader
net.eval()
split_comber = data_loader.dataset.split_comber
for i_name, (data, target, coord, nzhw) in enumerate(data_loader):
    assert i_name ==0
    s = time.time()
    target = [np.asarray(t, np.float32) for t in target]
    lbb = target[0]
    nzhw = nzhw[0]
    name = data_loader.dataset.filenames[i_name].split('-')[0].split('/')[-1]
    shortname = name.split('_clean')[0]
    data = data[0][0]
    #print data
    coord = coord[0][0]
    isfeat = False
    if 'output_feature' in config:
        if config['output_feature']:
            isfeat = True
    n_per_run = n_gpu
    print(data.size())
    splitlist = range(0,len(data)+1,n_gpu)
    if splitlist[-1]!=len(data):
        splitlist.append(len(data))
    outputlist = []
    featurelist = []

    for i in range(len(splitlist)-1):
        input = Variable(data[splitlist[i]:splitlist[i+1]], volatile = True)#.cuda()
        inputcoord = Variable(coord[splitlist[i]:splitlist[i+1]], volatile = True)#.cuda()
        if isfeat:
            output,feature = net(input,inputcoord)
            featurelist.append(feature.data.cpu().numpy())
        else:
            output = net(input,inputcoord)
        outputlist.append(output.data.cpu().numpy())
    output = np.concatenate(outputlist,0)
    output = split_comber.combine(output,nzhw=nzhw)
    if isfeat:
        feature = np.concatenate(featurelist,0).transpose([0,2,3,4,1])[:,:,:,:,:,np.newaxis]
        feature = split_comber.combine(feature,sidelen)[...,0]

    thresh = -3
    print output.shape
    pbb,mask = get_pbb(output,thresh,ismask=True)
    if isfeat:
        feature_selected = feature[mask[0],mask[1],mask[2]]
        np.save(os.path.join(save_dir, shortname+'_feature.npy'), feature_selected)
        #tp,fp,fn,_ = acc(pbb,lbb,0,0.1,0.1)
        #print([len(tp),len(fp),len(fn)])
    print([i_name,shortname])
    e = time.time()
    print pbb   
    np.save(os.path.join(save_dir, shortname+'_pbb.npy'), pbb)
    np.save(os.path.join(save_dir, shortname+'_lbb.npy'), lbb)
end_time = time.time()


print('elapsed time is %3.2f seconds' % (end_time - start_time))
print
print

NameError: name 'nod_net' is not defined

In [3]:
pbb.shape

NameError: name 'pbb' is not defined

In [4]:
output.shape

NameError: name 'output' is not defined

In [ ]:
coord

In [4]:
output = output
thresh = -3
ismask = True

In [5]:
output.shape

(72, 72, 72, 3, 5)

In [6]:
stride =4
anchors = np.array(config['anchors'])
output = np.copy(output)
offset = (float(stride) - 1) / 2
output_size = output.shape
oz = np.arange(offset, offset + stride * (output_size[0] - 1) + 1, stride)
oh = np.arange(offset, offset + stride * (output_size[1] - 1) + 1, stride)
ow = np.arange(offset, offset + stride * (output_size[2] - 1) + 1, stride)

In [7]:
oz

array([   1.5,    5.5,    9.5,   13.5,   17.5,   21.5,   25.5,   29.5,
         33.5,   37.5,   41.5,   45.5,   49.5,   53.5,   57.5,   61.5,
         65.5,   69.5,   73.5,   77.5,   81.5,   85.5,   89.5,   93.5,
         97.5,  101.5,  105.5,  109.5,  113.5,  117.5,  121.5,  125.5,
        129.5,  133.5,  137.5,  141.5,  145.5,  149.5,  153.5,  157.5,
        161.5,  165.5,  169.5,  173.5,  177.5,  181.5,  185.5,  189.5,
        193.5,  197.5,  201.5,  205.5,  209.5,  213.5,  217.5,  221.5,
        225.5,  229.5,  233.5,  237.5,  241.5,  245.5,  249.5,  253.5,
        257.5,  261.5,  265.5,  269.5,  273.5,  277.5,  281.5,  285.5])

In [11]:
#data

In [19]:
output[:, :, :, :, 1] = oz.reshape((-1, 1, 1, 1)) + output[:, :, :, :, 1] * anchors.reshape((1, 1, 1, -1))

In [15]:
output[:, :, :, :, 1]* anchors

array([[[[ -2.76665622e-02,   3.98059376e-02,  -1.08116269e-01],
         [ -4.08408139e-02,  -5.00238463e-01,  -5.31579182e-02],
         [ -1.32849813e-02,  -2.16119550e-02,   3.77191231e-02],
         ..., 
         [  1.24454312e-01,  -1.14198469e+00,  -1.71253134e+00],
         [  9.31207091e-02,  -9.75319929e-01,  -2.36158848e+00],
         [  6.11376995e-02,  -1.30707934e+00,  -2.06701398e+00]],

        [[ -1.07276272e-01,  -6.92180470e-01,   6.13658465e-01],
         [ -8.48404039e-02,  -4.44177501e-01,   8.01360402e-01],
         [ -7.80937867e-02,  -7.02690706e-01,   3.94279081e-01],
         ..., 
         [  6.25089277e-03,  -1.26871280e+00,  -1.70053404e+00],
         [ -3.80719965e-02,  -1.39602989e+00,  -1.98639840e+00],
         [ -2.51008756e-02,  -1.39852814e+00,  -1.98615387e+00]],

        [[ -2.47183442e-02,   9.90961678e-02,   1.37547646e-01],
         [ -5.69915585e-02,  -4.92616184e-01,   4.42340970e-04],
         [ -1.45755615e-02,   2.42812559e-02,   2.106924

In [16]:
output[:, :, :, :, 1] * anchors.reshape((1, 1, 1, -1))

array([[[[ -2.76665622e-02,   3.98059376e-02,  -1.08116269e-01],
         [ -4.08408139e-02,  -5.00238463e-01,  -5.31579182e-02],
         [ -1.32849813e-02,  -2.16119550e-02,   3.77191231e-02],
         ..., 
         [  1.24454312e-01,  -1.14198469e+00,  -1.71253134e+00],
         [  9.31207091e-02,  -9.75319929e-01,  -2.36158848e+00],
         [  6.11376995e-02,  -1.30707934e+00,  -2.06701398e+00]],

        [[ -1.07276272e-01,  -6.92180470e-01,   6.13658465e-01],
         [ -8.48404039e-02,  -4.44177501e-01,   8.01360402e-01],
         [ -7.80937867e-02,  -7.02690706e-01,   3.94279081e-01],
         ..., 
         [  6.25089277e-03,  -1.26871280e+00,  -1.70053404e+00],
         [ -3.80719965e-02,  -1.39602989e+00,  -1.98639840e+00],
         [ -2.51008756e-02,  -1.39852814e+00,  -1.98615387e+00]],

        [[ -2.47183442e-02,   9.90961678e-02,   1.37547646e-01],
         [ -5.69915585e-02,  -4.92616184e-01,   4.42340970e-04],
         [ -1.45755615e-02,   2.42812559e-02,   2.106924

In [17]:
oz.reshape((-1, 1, 1, 1))

array([[[[   1.5]]],


       [[[   5.5]]],


       [[[   9.5]]],


       [[[  13.5]]],


       [[[  17.5]]],


       [[[  21.5]]],


       [[[  25.5]]],


       [[[  29.5]]],


       [[[  33.5]]],


       [[[  37.5]]],


       [[[  41.5]]],


       [[[  45.5]]],


       [[[  49.5]]],


       [[[  53.5]]],


       [[[  57.5]]],


       [[[  61.5]]],


       [[[  65.5]]],


       [[[  69.5]]],


       [[[  73.5]]],


       [[[  77.5]]],


       [[[  81.5]]],


       [[[  85.5]]],


       [[[  89.5]]],


       [[[  93.5]]],


       [[[  97.5]]],


       [[[ 101.5]]],


       [[[ 105.5]]],


       [[[ 109.5]]],


       [[[ 113.5]]],


       [[[ 117.5]]],


       [[[ 121.5]]],


       [[[ 125.5]]],


       [[[ 129.5]]],


       [[[ 133.5]]],


       [[[ 137.5]]],


       [[[ 141.5]]],


       [[[ 145.5]]],


       [[[ 149.5]]],


       [[[ 153.5]]],


       [[[ 157.5]]],


       [[[ 161.5]]],


       [[[ 165.5]]],


       [[[ 169.5]]],


       [[[ 

In [21]:
output[:, :, :, :, 1]

array([[[[  1.47233344e+00,   1.53980594e+00,   1.39188373e+00],
         [  1.45915919e+00,   9.99761537e-01,   1.44684208e+00],
         [  1.48671502e+00,   1.47838804e+00,   1.53771912e+00],
         ..., 
         [  1.62445431e+00,   3.58015314e-01,  -2.12531336e-01],
         [  1.59312071e+00,   5.24680071e-01,  -8.61588478e-01],
         [  1.56113770e+00,   1.92920655e-01,  -5.67013979e-01]],

        [[  1.39272373e+00,   8.07819530e-01,   2.11365847e+00],
         [  1.41515960e+00,   1.05582250e+00,   2.30136040e+00],
         [  1.42190621e+00,   7.97309294e-01,   1.89427908e+00],
         ..., 
         [  1.50625089e+00,   2.31287196e-01,  -2.00534038e-01],
         [  1.46192800e+00,   1.03970110e-01,  -4.86398399e-01],
         [  1.47489912e+00,   1.01471856e-01,  -4.86153871e-01]],

        [[  1.47528166e+00,   1.59909617e+00,   1.63754765e+00],
         [  1.44300844e+00,   1.00738382e+00,   1.50044234e+00],
         [  1.48542444e+00,   1.52428126e+00,   1.710692

In [22]:
output[:, :, :, :, 2] = oh.reshape((1, -1, 1, 1)) + output[:, :, :, :, 2] * anchors.reshape((1, 1, 1, -1))
output[:, :, :, :, 3] = ow.reshape((1, 1, -1, 1)) + output[:, :, :, :, 3] * anchors.reshape((1, 1, 1, -1))
output[:, :, :, :, 4] = np.exp(output[:, :, :, :, 4]) * anchors.reshape((1, 1, 1, -1))

In [23]:
mask = output[..., 0] > thresh

In [24]:
xx,yy,zz,aa = np.where(mask)

In [25]:
output = output[xx,yy,zz,aa]

In [27]:
output = output[np.argsort(-output[:, 0])]

In [28]:
output

array([[   8.44247723,  160.85776174,   80.96628187,   25.0723585 ,
           7.8915176 ],
       [   7.96633959,  158.58887888,   80.56826653,   25.03590505,
           8.15776766],
       [   7.06651497,  160.77239507,   78.52150597,   24.98223114,
           8.09625931],
       ..., 
       [  -2.99373269,  104.9343039 ,   30.03030759,   57.78107595,
          29.25612744],
       [  -2.99599457,   73.95736291,   84.40532479,  213.34927623,
          60.19464309],
       [  -2.99652886,  102.6238334 ,  137.52942529,  156.57512885,
          21.33820906]])

In [29]:
bboxes = [output[0]]

In [31]:
def iou(box0, box1):
    
    r0 = box0[3] / 2
    s0 = box0[:3] - r0
    e0 = box0[:3] + r0

    r1 = box1[3] / 2
    s1 = box1[:3] - r1
    e1 = box1[:3] + r1

    overlap = []
    for i in range(len(s0)):
        overlap.append(max(0, min(e0[i], e1[i]) - max(s0[i], s1[i])))

    intersection = overlap[0] * overlap[1] * overlap[2]
    union = box0[3] * box0[3] * box0[3] + box1[3] * box1[3] * box1[3] - intersection
    return intersection / union

In [34]:
for j in range(len(bboxes)):
        bbox = output[i]
        flag = 1
        if iou(bbox[1:5], bboxes[j][1:5]) >= 0.5:
                flag = -1
                break
        if flag == 1:
            bboxes.append(bbox)


In [35]:
bboxes

[array([   8.44247723,  160.85776174,   80.96628187,   25.0723585 ,
           7.8915176 ]),
 array([   6.22056389,  158.21139939,   80.69530809,   22.64749961,
           8.17352259])]